In [13]:
# submission_rate = [20, 40, 60, 80, 100, 200, 400, 600, 800, 1000, 2000, 4000, 6000, 8000, 10000, 20000, 40000, 60000, 80000, 100000]
num_clinets = [1, 2, 3, 4]
from pathlib import Path
import os

# for i in range(len(submission_rate)):
#     filename = "submit_rate_intra_{}.log".format(submission_rate[i])
#     grepped_file_0 = f"../WAN-without-dom/receiver_submit_rate_{submission_rate[i]}_0.log"
#     grepped_file_1 = f"../WAN-without-dom/receiver_submit_rate_{submission_rate[i]}_1.log"
    
    
#     # Use os.system to execute the grep command
#     os.system(f'grep "Forwarding request " {grepped_file_0} {grepped_file_1}  > {filename}')
#     print("filename:", filename)

for i in range(len(num_clinets)):
    filename = "num_clients_inter_{}.log".format(num_clinets[i])
    grepped_file_0 = f"../logs/{num_clinets[i]}_clients_receiver_submit_rate_4000_0.log"
    grepped_file_1 = f"../logs/{num_clinets[i]}_clients_receiver_submit_rate_4000_1.log"
    os.system(f'grep "Forwarding request " {grepped_file_0} {grepped_file_1}  > {filename}')
    print("filename:", filename)

    

filename: num_clients_inter_1.log
filename: num_clients_inter_2.log
filename: num_clients_inter_3.log
filename: num_clients_inter_4.log


In [14]:
import re
def parse_int_tag(line, name):
    match = re.search(f"{name}=([0-9]*)", line)
    if match:
        return int(match.group(1))
    else:
        return 0

def parse_line(line):
    deadline = parse_int_tag(line, "deadline")
    now = parse_int_tag(line, "now")
    id = parse_int_tag(line, "r_id")
    proxy_id = parse_int_tag(line, "c_id")
    proxy_seq = parse_int_tag(line, "c_seq")

    return (id, proxy_id, proxy_seq)    

In [15]:
from collections import defaultdict

# Get edit distance between 2 sequences from receivers
def req_eq(r1, r2):
    return r1[0] == r2[0] and r1[1] == r2[1]

def edit_dist(s1, s2):
    dp = [ ([-1] * (len(s2) + 1)) for _ in range(len(s1) + 1)]

    dp[0][0] = 0

    for i in range(1, len(dp)):
        dp[i][0] = i
        
    for j in range(1, len(dp[0])):
        dp[0][j] = j
        
    for i in range(1, len(dp)):
        for j in range(1, len(dp[i])):
            # Change req at position i/j
            if req_eq(s1[i-1], s2[j-1]):
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + dp[i - 1][j - 1]

            # TODO swaps?
            dp[i][j] = min(dp[i][j], 1 + dp[i - 1][j], 1 + dp[i][j - 1])
    
    return dp[-1][-1]


# Longest Increasing Subsequence
def lis(s1, s2, truncation = None):
    # use s1 as truth
    if truncation:
        s1 = s1[:truncation]
        s2 = s2[:truncation]
    
    entry_to_idx = defaultdict(int)
    for i, x in enumerate(s1):
        x = (x[0], x[1])
        entry_to_idx[x] = i

    seq = [entry_to_idx[(x[0], x[1])] for x in s2]    

    dp = []

    for i in range(len(seq)):
        dp.append(1)
        for j in range(i):
            if seq[i] > seq[j]: 
                dp[i] = max(dp[j] + 1, dp[i])

    return max(dp)

def lis_multiple(*seqs):
    s_true = seqs[0]
    n = len(s_true)

    ret = []
    for s in seqs[1:]:
        ret.append(lis(s_true, s) / n)

    return ret

# Simpler metric, which perecentage of requests are the same?
def matching_reqs(*seqs):
    n = len(seqs[0])
    ret = []
    
    for entries in zip(*seqs):
        match = True
        for (x, y) in zip(entries, entries[1:]):
            if not req_eq(x, y): match = False

        ret.append(match)
        
    return ret

In [16]:
from collections import defaultdict
import bisect

def efficient_lis(s1, s2, truncation=None):
    # Use s1 as the ground truth
    if truncation:
        s1 = s1[:truncation]
        s2 = s2[:truncation]
    
    entry_to_idx = defaultdict(int)
    
    for i, x in enumerate(s1):
        x = (x[0], x[1])
        entry_to_idx[x] = i
    
    # Map entries in s2 to their indices in s1
    seq = [entry_to_idx[(x[0], x[1])] for x in s2 if (x[0], x[1]) in entry_to_idx]
    
    # Now compute LIS on the mapped sequence using binary search for efficiency
    lis = []
    
    for num in seq:
        pos = bisect.bisect_left(lis, num)
        if pos == len(lis):
            lis.append(num)
        else:
            lis[pos] = num
    
    return len(lis)

In [11]:
import itertools


requests={}
for i in range(len(num_clinets)):
    requests = {}
    
    filename = "num_clients_inter_{}.log".format(num_clinets[i])
    print("filename is: ", filename)
    for line in open(filename):
        
        (id, *req) = parse_line(line)

        if id not in requests:
            requests[id] = []
        
        requests[id].append(tuple(req))
    
        
    reqs = matching_reqs(*[v for v in requests.values()])
    
    print(f"Total number of requests: {len(reqs)}")
    print(f"Total proportion of requests the same: {sum(reqs) / len(reqs):.2f}")

filename is:  submit_rate_intra_20.log
Total number of requests: 822
Total proportion of requests the same: 0.58
filename is:  submit_rate_intra_40.log
Total number of requests: 1595
Total proportion of requests the same: 0.36
filename is:  submit_rate_intra_60.log
Total number of requests: 2434
Total proportion of requests the same: 0.20
filename is:  submit_rate_intra_80.log
Total number of requests: 3074
Total proportion of requests the same: 0.18
filename is:  submit_rate_intra_100.log
Total number of requests: 3798
Total proportion of requests the same: 0.15
filename is:  submit_rate_intra_200.log
Total number of requests: 7173
Total proportion of requests the same: 0.10
filename is:  submit_rate_intra_400.log
Total number of requests: 12963
Total proportion of requests the same: 0.10
filename is:  submit_rate_intra_600.log
Total number of requests: 17475
Total proportion of requests the same: 0.10
filename is:  submit_rate_intra_800.log
Total number of requests: 21173
Total propo

In [12]:
for i in range(len(num_clinets)):
    requests = {}
    
    filename = "num_clients_inter_{}.log".format(num_clinets[i])
    print("filename is: ", filename)
    for line in open(filename):
        
        (id, *req) = parse_line(line)

        if id not in requests:
            requests[id] = []
        
        requests[id].append(tuple(req))
    
    total_len = len(requests[0])
    print("total len: ", total_len)
    print(efficient_lis(requests[0], requests[1]))

filename is:  submit_rate_intra_20.log
total len:  822
674
filename is:  submit_rate_intra_40.log
total len:  1595
1191
filename is:  submit_rate_intra_60.log
total len:  2434
1673
filename is:  submit_rate_intra_80.log
total len:  3074
2101
filename is:  submit_rate_intra_100.log
total len:  3798
2512
filename is:  submit_rate_intra_200.log
total len:  7173
4418
filename is:  submit_rate_intra_400.log
total len:  12963
7536
filename is:  submit_rate_intra_600.log
total len:  17475
9948
filename is:  submit_rate_intra_800.log
total len:  21173
11869
filename is:  submit_rate_intra_1000.log
total len:  23983
13386
filename is:  submit_rate_intra_2000.log
total len:  79936
45170
filename is:  submit_rate_intra_4000.log
total len:  159716
88305
filename is:  submit_rate_intra_6000.log
total len:  224582
120203
filename is:  submit_rate_intra_8000.log
total len:  234134
106973
filename is:  submit_rate_intra_10000.log
total len:  240801
100972
filename is:  submit_rate_intra_20000.log
tota